In [4]:
# Overlapping Sequence Detector FSM (without class)
# Detects the sequence "1011" in a binary input stream

def process_bit(bit, state, sequence):
    seq = sequence
    detected = False

    if bit == seq[state]:
        state += 1
        if state == len(seq):
            detected = True
            # Handle overlap: move to the longest suffix that matches a prefix
            for next_state in range(len(seq) - 1, -1, -1):
                if seq[:next_state] == seq[state - next_state:state]:
                    state = next_state
                    break
    else:
        # If mismatch, find the longest prefix that matches suffix + current bit
        found = False
        for next_state in range(state, -1, -1):
            if seq[:next_state] == (seq[state - next_state:state] + bit)[-next_state:]:
                state = next_state
                found = True
                break
        if not found:
            state = 1 if bit == seq[0] else 0
    
    return state, detected


def process_stream(stream, sequence="1011"):
    detections = []
    state = 0
    for i, bit in enumerate(stream):
        state, detected = process_bit(bit, state, sequence)
        if detected:
            detections.append(i)
    return detections


# Example usage
test_stream = "1011011001011"
detections = process_stream(test_stream, "1011")
print("Detections at positions:", detections)


Detections at positions: [3, 6, 12]


In [12]:
def generate_verilog(sequence, module_name="seq_detector"):
    n = len(sequence)
    num_states = n + 1   # S0..Sn
    bits = (num_states-1).bit_length()  # binary width

    # Generate state names
    states = [f"S{i}" for i in range(num_states)]

    verilog = []
    verilog.append(f"module {module_name}(input clk, input reset, input din, output reg detected);\n")

    # Parameter state encoding
    verilog.append("  // State encoding")
    for i, s in enumerate(states):
        verilog.append(f"  parameter {s} = {bits}'d{i};")

    verilog.append(f"\n  reg [{bits-1}:0] state, next_state;\n")

    # Sequential block
    verilog.append("  always @(posedge clk or posedge reset) begin")
    verilog.append("    if (reset)")
    verilog.append(f"      state <= {states[0]};")
    verilog.append("    else")
    verilog.append("      state <= next_state;")
    verilog.append("  end\n")

    # Next-state logic
    verilog.append("  always @(*) begin")
    verilog.append("    detected = 0;")
    verilog.append("    case (state)")

    for i in range(num_states):
        verilog.append(f"      {states[i]}: begin")
        if i == n:
            # Final state → detection
            verilog.append("        detected = 1;")
            # Overlap handling
            for b in ['0','1']:
                suffix = sequence[1:] + b
                next_state = 0
                for k in range(n,0,-1):
                    if sequence[:k] == suffix[-k:]:
                        next_state = k
                        break
                verilog.append(f"        if (din == 1'b{b}) next_state = {states[next_state]};")
        else:
            for b in ['0','1']:
                if b == sequence[i]:
                    verilog.append(f"        if (din == 1'b{b}) next_state = {states[i+1]};")
                else:
                    # Fallback
                    suffix = sequence[:i] + b
                    next_state = 0
                    for k in range(i,0,-1):
                        if sequence[:k] == suffix[-k:]:
                            next_state = k
                            break
                    verilog.append(f"        if (din == 1'b{b}) next_state = {states[next_state]};")
        verilog.append("      end")

    verilog.append("      default: next_state = S0;")
    verilog.append("    endcase")
    verilog.append("  end\nendmodule")

    return "\n".join(verilog)


In [13]:
sequence = "101110"
module_name = "seq101110_detector"
verilog_code = generate_verilog(sequence, module_name)

# Save to .v file
with open(f"{module_name}.v", "w") as f:
    f.write(verilog_code)


In [ ]:
def generate_testbench(sequence, module_name="seq_detector", tb_name="tb_seq_detector", test_stream="1011011001011"):
    tb = []
    tb.append(f"module {tb_name};")
    tb.append("  reg clk, reset, din;")
    tb.append("  wire detected;")
    tb.append(f"  {module_name} uut (.clk(clk), .reset(reset), .din(din), .detected(detected));")
    tb.append("")
    # Clock generation
    tb.append("  initial begin")
    tb.append("    clk = 0;")
    tb.append("    forever #5 clk = ~clk;")
    tb.append("  end")
    tb.append("")
    # Stimulus
    tb.append("  initial begin")
    tb.append("    $dumpfile(\"wave.vcd\");")
    tb.append("    $dumpvars(0, uut);")
    tb.append("    reset = 1; din = 0; #12;")
    tb.append("    reset = 0;")
    tb.append(f"    // Test stream: {test_stream}")
    for bit in test_stream:
        tb.append(f"    din = 1'b{bit}; #10;")
    tb.append("    #20; $finish;")
    tb.append("  end")
    tb.append("endmodule")
    return "\n".join(tb)


In [16]:
sequence = "100111"
module_name = "seq100111_detector"

verilog_code = generate_verilog(sequence, module_name)
testbench_code = generate_testbench(sequence, module_name, "tb_seq100111")

with open(f"{module_name}.v", "w") as f:
    f.write(verilog_code)

with open(f"{module_name}_tb.v", "w") as f:
    f.write(testbench_code)

print(f"Generated {module_name}.v and {module_name}_tb.v")

Generated seq100111_detector.v and seq100111_detector_tb.v
